<a href="https://colab.research.google.com/github/sota1111/DL_Zero3/blob/main/DL_Zero3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

import os, sys
os.chdir('/content/drive/MyDrive/DL_Zero')

!git clone https://github.com/sota1111/DL_Zero3.git

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'DL_Zero3' already exists and is not an empty directory.


In [6]:
!git config --global user.email "sota.moro@gmail.com"
!git config --global user.name "sota1111"

In [7]:
!git add DL_Zero3_1.ipynb
!git commit -m "Added new file."

fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


# stage1 微分を自動で求める

## step1 箱としての変数

In [ ]:
class Variable:
  def __init__(self, data):
    self.data = data

In [ ]:
import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


In [ ]:
x.data = np.array(2.0)
print(x.data)

2.0


In [ ]:
import numpy as np
x = np.array(1)
x.ndim

0

In [ ]:
x = np.array([1, 2, 3])
x.ndim

1

In [ ]:
x = np.array([[1,2,3],[4,5,6]])
x.ndim

2

## step2 変数を生み出す関数

In [ ]:
class Function:
  def __call__(self, input):
    x = input.data #データを取り出す
    y = x ** 2
    output = Variable(y) #Varibleとして取り出す
    return output

In [ ]:
x = Variable(np.array(10))
f = Function()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [ ]:
class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    return output

  def forward(self, x):
    raise NotImplementedError()

In [ ]:
class Square(Function):
  def forward(self, x):
    return x ** 2

In [ ]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


## step3 関数の連結